In [1]:
# 设计一个程序，用于获取pathway的信息
#读取pathway的信息
import os
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup

In [2]:
df_pathway = pd.read_csv('c2.cp.kegg_medicus.v2023.2.Hs.symbols_INFO.csv')
# 对pathway_name进行去重，打印
df_pathway['pathway_class'].drop_duplicates()

0      nt06252 Mitochondrial ROS formation.
1       nt06251 CYP-mediated ROS formation.
2                    nt06210 ERK signaling.
3             nt06250 DNA adduct formation.
4       nt06227 Nuclear receptor signaling.
                       ...                 
495                                       .
512                nt06272 Prostate cancer.
524               nt06360 Cushing syndrome.
538           nt06269 Basal cell carcinoma.
554             nt06461 Huntington disease.
Name: pathway_class, Length: 111, dtype: object

In [6]:
df_pathway = pd.read_csv('c2.cp.kegg.v2023.1.Hs.symbols_info.csv')
print(df_pathway.shape)
print(df_pathway.head())

(186, 4)
                                        pathway_name  pathway_id  \
0                         KEGG_N_GLYCAN_BIOSYNTHESIS         NaN   
1                      KEGG_OTHER_GLYCAN_DEGRADATION         NaN   
2                         KEGG_O_GLYCAN_BIOSYNTHESIS         NaN   
3                 KEGG_GLYCOSAMINOGLYCAN_DEGRADATION         NaN   
4  KEGG_GLYCOSAMINOGLYCAN_BIOSYNTHESIS_KERATAN_SU...         NaN   

   pathway_type  pathway_class  
0           NaN            NaN  
1           NaN            NaN  
2           NaN            NaN  
3           NaN            NaN  
4           NaN            NaN  


In [7]:
import re
import requests
from bs4 import BeautifulSoup

def get_pathway_info(pathway_name):
    url = f'https://www.gsea-msigdb.org/gsea/msigdb/human/geneset/{pathway_name}.html'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find("table",class_="lists4 human")
    trs = table.find_all("tr")
    pathway_id = ''
    pathway_type = ''
    pathway_class = ''
    pattern = r'Pathway ID: (.+)\. Pathway type: (.+)\. Pathway class: (.+)'

    for tr in trs:
        if "Full description or abstract" in tr.text:
            match = re.search(pattern, tr.find("td").text)
            pathway_id = match.group(1)
            pathway_type = match.group(2)
            pathway_class = match.group(3)
            break
    return pathway_id, pathway_type, pathway_class

In [8]:
# 循环df_pathway的pathway_name，获取pathway的信息
pathway_info = []
count = 0
for pathway_name in df_pathway['pathway_name']:
    print(count,pathway_name)
    count += 1
    pathway_id, pathway_type, pathway_class = get_pathway_info(pathway_name)
    pathway_info.append([pathway_id, pathway_type, pathway_class])

0 KEGG_N_GLYCAN_BIOSYNTHESIS


AttributeError: 'NoneType' object has no attribute 'group'

In [25]:
df_pathway_info = pd.DataFrame(pathway_info, columns=['pathway_id', 'pathway_type', 'pathway_class'])
df_pathway_info['pathway_name'] = df_pathway['pathway_name']
df_pathway_info.head()

,pathway_id,pathway_type,pathway_class,pathway_name
0,N01394,Env factor,nt06252 Mitochondrial ROS formation.,KEGG_MEDICUS_ENV_FACTOR_ARSENIC_TO_ELECTRON_TR...
1,N01401,Env factor,nt06251 CYP-mediated ROS formation.,KEGG_MEDICUS_ENV_FACTOR_BENZO_A_PYRENRE_TO_CYP...
2,N01352,Env factor,nt06210 ERK signaling.,KEGG_MEDICUS_ENV_FACTOR_BPA_TO_RAS_ERK_SIGNALI...
3,N01386,Env factor,nt06250 DNA adduct formation.,KEGG_MEDICUS_ENV_FACTOR_DCE_TO_DNA_ADDUCTS
4,N01364,Env factor,nt06227 Nuclear receptor signaling.,KEGG_MEDICUS_ENV_FACTOR_E2_TO_NUCLEAR_INITIATE...


In [12]:
pathway_name = "KEGG_MEDICUS_ENV_FACTOR_ARSENIC_TO_ELECTRON_TRANSFER_IN_COMPLEX_IV"
url = f'https://www.gsea-msigdb.org/gsea/msigdb/human/geneset/{pathway_name}.html'

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
#lists4 human
table = soup.find("table",class_="lists4 human")
# 找到tr里的内容包含Full description or abstract的tr
trs = table.find_all("tr")
pathway_id = ''
pathway_type = ''
pathway_class = ''
pattern = r'Pathway ID: (.+)\. Pathway type: (.+)\. Pathway class: (.+)'

for tr in trs:
    if "Full description or abstract" in tr.text:
        match = re.search(pattern, tr.find("td").text)
        pathway_id = match.group(1)
        pathway_type = match.group(2) 
        pathway_class = match.group(3)
        # Pathway_ID = tr.find("td").text
        # print(tr.text)
        break
